# <p><span style="color:blue">  Activity 2 - Notebook 1 - Soccer data  </span></p>

# <p><span style="color:blue">  Procedure followed : <br/> 1. Preprocessed and created Match csv. <br/> 2. Preprocessed and created Player csv. <br/> 3. Imported csv and answered questions. </span></p>

# <p><span style="color:blue"> This notebook pre processes data and creates csv</span></p>

In [2]:
library(dplyr)

In [3]:
# my_db <- src_sqlite("/media/23e97447-9196-4c8e-be28-b24a49d2548e/home/jayaraj/Study/DIC/database.sqlite")
my_db <- src_sqlite("./database.sqlite")

In [4]:
# my_db #src:  sqlite 3.11.1 [/media/23e97447-9196-4c8e-be28-b24a49d2548e/home/jayaraj/Study/DIC/database.sqlite]
# tbls: Country, League, Match, Player, Player_Attributes, sqlite_sequence, Team,
#   Team_Attributes

In [5]:
# country_tbl <- tbl(my_db, sql("SELECT * FROM Country"))
# league_tbl <- tbl(my_db, sql("SELECT * FROM League"))
match_tbl <- tbl(my_db, sql("SELECT * FROM Match"))
player_tbl <- tbl(my_db, sql("SELECT * FROM Player"))
player_attr_tbl <- tbl(my_db, sql("SELECT * FROM Player_Attributes"))
# sqlite_sequence_tbl <- tbl(my_db, sql("SELECT * FROM sqlite_sequence"))
team_tbl <- tbl(my_db, sql("SELECT * FROM Team"))
team_Attributes_tbl <- tbl(my_db, sql("SELECT * FROM Team_Attributes"))

## <p><span style="color:blue">  Preprocessing team related columns. Once all the columns are ready, csv will be created.  </span></p>

In [6]:
team_tbl
nrow(data.frame(team_tbl))
team_Attributes_tbl
nrow(data.frame(team_Attributes_tbl))

Source:   query [?? x 5]
Database: sqlite 3.11.1 [/media/23e97447-9196-4c8e-be28-b24a49d2548e/home/jayaraj/Study/DIC/database.sqlite]

      id team_api_id team_fifa_api_id    team_long_name team_short_name
   <int>       <int>            <int>             <chr>           <chr>
1      1        9987              673          KRC Genk             GEN
2      2        9993              675      Beerschot AC             BAC
3      3       10000            15005  SV Zulte-Waregem             ZUL
4      4        9994             2007  Sporting Lokeren             LOK
5      5        9984             1750 KSV Cercle Brugge             CEB
6      6        8635              229    RSC Anderlecht             AND
7      7        9991              674          KAA Gent             GEN
8      8        9998             1747         RAEC Mons             MON
9      9        7947               NA     FCV Dender EH             DEN
10    10        9985              232 Standard de Liège             STL
#

[1] 299

Source:   query [?? x 25]
Database: sqlite 3.11.1 [/media/23e97447-9196-4c8e-be28-b24a49d2548e/home/jayaraj/Study/DIC/database.sqlite]

      id team_fifa_api_id team_api_id                date buildUpPlaySpeed
   <int>            <int>       <int>               <chr>            <int>
1      1              434        9930 2010-02-22 00:00:00               60
2      2              434        9930 2014-09-19 00:00:00               52
3      3              434        9930 2015-09-10 00:00:00               47
4      4               77        8485 2010-02-22 00:00:00               70
5      5               77        8485 2011-02-22 00:00:00               47
6      6               77        8485 2012-02-22 00:00:00               58
7      7               77        8485 2013-09-20 00:00:00               62
8      8               77        8485 2014-09-19 00:00:00               58
9      9               77        8485 2015-09-10 00:00:00               59
10    10              614        8576 2

[1] 1458

In [7]:
temp <- tbl(my_db,sql("SELECT Team.team_api_id
from Team LEFT JOIN Team_Attributes ON Team_Attributes.team_api_id = Team.team_api_id GROUP BY Team.team_api_id"))
nrow(data.frame(temp))

[1] 299

In [8]:
#team attributes with sum of build up, chance creation and defence. 
team_attr_with_sum <- tbl(my_db, sql("SELECT Team.team_api_id,team_long_name,MAX(buildUpPlaySpeed+buildUpPlayPassing+buildUpPlayDribbling) as sum_buildup,
MAX(chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting) as sum_chance_creation,
MAX(defencePressure+defenceAggression+defenceTeamWidth) as sum_defence
FROM Team LEFT JOIN Team_Attributes ON Team_Attributes.team_api_id = Team.team_api_id GROUP BY Team.team_api_id,team_short_name "))


In [9]:
team_with_match_goals <- tbl(my_db,sql("select team_api_id, sum(goals) as sum_goals from (select Team.team_api_id,home_team_goal as goals from Team INNER JOIN
Match ON Team.team_api_id=home_team_api_id UNION ALL select Team.team_api_id,away_team_goal as goals from Team LEFT JOIN
Match ON Team.team_api_id=away_team_api_id) GROUP BY team_api_id"))

In [10]:
team_with_match_goals
nrow(data.frame(team_with_match_goals))
team_attr_with_sum
nrow(data.frame(team_attr_with_sum))

Source:   query [?? x 2]
Database: sqlite 3.11.1 [/media/23e97447-9196-4c8e-be28-b24a49d2548e/home/jayaraj/Study/DIC/database.sqlite]

   team_api_id sum_goals
         <int>     <int>
1         1601       280
2         1773       126
3         1957       287
4         2033       138
5         2182       372
6         2183       184
7         2186       235
8         4049        35
9         4064        27
10        4087       180
# ... with more rows

[1] 299

Source:   query [?? x 5]
Database: sqlite 3.11.1 [/media/23e97447-9196-4c8e-be28-b24a49d2548e/home/jayaraj/Study/DIC/database.sqlite]

   team_api_id           team_long_name sum_buildup sum_chance_creation
         <int>                    <chr>       <int>               <int>
1         1601             Ruch Chorzów         139                 186
2         1773      Oud-Heverlee Leuven         153                 157
3         1957    Jagiellonia Białystok         158                 163
4         2033           S.C. Olhanense         147                 165
5         2182              Lech Poznań         176                 196
6         2183              P. Warszawa          NA                 180
7         2186                 Cracovia         125                 168
8         4049                   Tubize          NA                  NA
9         4064                 Feirense          NA                  NA
10        4087 Évian Thonon Gaillard FC         156                 180
#

[1] 299

###  <p><span style="color:blue">team_all_final table/df has all the columns needed for answering the questions. It will be written to csv.</span></p>

In [11]:
team_all_final <- inner_join(x =team_attr_with_sum,y =team_with_match_goals,by =  "team_api_id" )
(data.frame(team_all_final))
nrow(data.frame(team_all_final))

team_api_id,team_long_name,sum_buildup,sum_chance_creation,sum_defence,sum_goals
1601,Ruch Chorzów,139,186,175,280
1773,Oud-Heverlee Leuven,153,157,137,126
1957,Jagiellonia Białystok,158,163,210,287
2033,S.C. Olhanense,147,165,155,138
2182,Lech Poznań,176,196,160,372
2183,P. Warszawa,NA,180,175,184
2186,Cracovia,125,168,205,235
4049,Tubize,NA,NA,NA,35
4064,Feirense,NA,NA,NA,27
4087,Évian Thonon Gaillard FC,156,180,165,180


[1] 299

In [12]:
nrow(data.frame(team_attr_with_sum))

[1] 299

In [13]:
# select(.data = team_attr_with_sum,team_api_id,buildUpPlaySpeed,buildUpPlayPassing,sum_buildup)#,chanceCreationPassing,chanceCreationCrossing,sum_chance_creation)
# # data.frame(select(.data = team_attr_with_sum,team_api_id,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,sum_chance_creation))
# # select(.data = team_attr_with_sum,team_api_id,defencePressure,defenceAggression,defenceTeamWidth,sum_defence)

In [14]:
write.csv(file='/media/23e97447-9196-4c8e-be28-b24a49d2548e/home/jayaraj/Study/DIC/lab2/soccer_csv/team_final.csv', x=team_all_final)

## <p><span style="color:blue">  ----- csv for TEAM created ----  </span></p>

## <p><span style="color:blue">  Preprocessing player related columns. Once all the columns are ready, csv will be created.  </span></p>

In [15]:
player_tbl_with_app <- tbl(my_db, sql("SELECT player_api_id,player_name,COUNT(player_name) FROM (SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.home_player_1 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.home_player_2 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.home_player_3 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.home_player_4 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.home_player_5 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.home_player_6 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.home_player_7 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.home_player_8 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.home_player_9 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.home_player_10 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.home_player_11 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.away_player_1 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.away_player_2 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.away_player_3 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.away_player_4 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.away_player_5 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.away_player_6 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.away_player_7 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.away_player_8 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.away_player_9 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.away_player_10 UNION ALL SELECT player_api_id,player_name FROM Player,Match WHERE Player.player_api_id
= Match.away_player_11) GROUP BY player_api_id,player_name"))
# SELECT * FROM Table1, Table2 WHERE Table1.id = Table2.id_table1 GROUP BY Table1.data1

In [16]:
player_tbl_with_app_and_league <- tbl(my_db, sql("SELECT player_api_id,player_name,COUNT(player_name) as appearances,COUNT(DISTINCT(league_id
)) as league_count FROM (SELECT player_api_id,player_name,league_id FROM Player,Match WHERE Player.player_api_id
= Match.home_player_1 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.home_player_2 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.home_player_3 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.home_player_4 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.home_player_5 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.home_player_6 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.home_player_7 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.home_player_8 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.home_player_9 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.home_player_10 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.home_player_11 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.away_player_1 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.away_player_2 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.away_player_3 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.away_player_4 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.away_player_5 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.away_player_6 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.away_player_7 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.away_player_8 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.away_player_9 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.away_player_10 UNION ALL SELECT player_api_id,player_name,league_id  FROM Player,Match WHERE Player.player_api_id
= Match.away_player_11) GROUP BY player_api_id,player_name"))
# SELECT * FROM Table1, Table2 WHERE Table1.id = Table2.id_table1 GROUP BY Table1.data1

In [17]:
nrow(data.frame(player_tbl))
nrow(data.frame(player_attr_tbl))

[1] 11060

[1] 183978

In [18]:
player_sum_gk_and_over <- tbl(my_db,sql("SELECT Player.player_api_id,overall_rating,
(gk_diving+gk_handling+gk_kicking+gk_positioning+gk_reflexes) as sum_gk from Player LEFT JOIN Player_Attributes ON 
Player.player_api_id = Player_Attributes.player_api_id"))

In [19]:
# player_tbl_with_app_and_league
player_appr_ovr_sum_gk_lg_cnt <- inner_join(x = player_sum_gk_and_over,y = player_tbl_with_app_and_league,by = "player_api_id" )

In [20]:
player_appr_ovr_sum_gk_lg_cnt

Source:   query [?? x 6]
Database: sqlite 3.11.1 [/media/23e97447-9196-4c8e-be28-b24a49d2548e/home/jayaraj/Study/DIC/database.sqlite]

   player_api_id overall_rating sum_gk       player_name appearances
           <int>          <int>  <int>             <chr>       <int>
1           2625             58     45 Patryk Rachwal,18          90
2           2625             58     45 Patryk Rachwal,18          90
3           2625             58     45 Patryk Rachwal,18          90
4           2625             58     45 Patryk Rachwal,18          90
5           2625             59     45 Patryk Rachwal,18          90
6           2625             60     45 Patryk Rachwal,18          90
7           2625             60    153 Patryk Rachwal,18          90
8           2625             61     45 Patryk Rachwal,18          90
9           2625             61     45 Patryk Rachwal,18          90
10          2625             61     45 Patryk Rachwal,18          90
# ... with more rows, and 1 more vari

In [21]:
nrow(data.frame(player_appr_ovr_sum_gk_lg_cnt))

[1] 183978

In [22]:
player_appr_ovr_sum_gk_lg_cnt_df <- data.frame(player_appr_ovr_sum_gk_lg_cnt)

In [23]:
write.csv(file='/media/23e97447-9196-4c8e-be28-b24a49d2548e/home/jayaraj/Study/DIC/lab2/soccer_csv/player_final.csv', x=player_appr_ovr_sum_gk_lg_cnt_df)

## <p><span style="color:blue">  CSV for Player related columns created.  </span></p>

## <p><span style="color:blue">  -----------Preprocessing done.-----------  </span></p>

## <p><span style="color:blue">----------Questions and answers in next notebook.---------</span></p>